# Sentencias

In [ ]:
# Cargar paquetes
import os
import pandas as pd
import requests

# Cargar variables de entorno en .env
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# Hola al API OAuth2
HOST = "http://127.0.0.1:8002"
respuesta = requests.get(HOST)
print(respuesta.status_code)
print(respuesta.text)

In [ ]:
# Autentificarse
parametros = {
    "username": os.getenv("USERNAME"),
    "password": os.getenv("PASSWORD"),
}
cabeceras_para_autentificar = {'content-type': "application/x-www-form-urlencoded"}
respuesta = requests.post(f"{HOST}/token", data=parametros, headers=cabeceras_para_autentificar)
print(respuesta.status_code)
cabeceras_para_api = None
if respuesta.status_code == 200:
    token = respuesta.json()['access_token']
    cabeceras_para_api = {'Authorization': 'Bearer ' + token}
print(cabeceras_para_api)

In [ ]:
# Consultar
parametros = { "fecha_desde": "2022-06-17" }
try:
    respuesta = requests.get(f"{HOST}/v1/sentencias", headers=cabeceras_para_api, params=parametros, timeout=12)
except requests.exceptions.RequestException as error:
    raise Exception("Parece que hay un error en la consulta: " + str(error))
if respuesta.status_code != 200:
    raise requests.HTTPError("Parece que ya caducó el token")
columns = ["distrito_nombre_corto", "autoridad_clave", "materia_tipo_juicio_descripcion", "sentencia", "sentencia_fecha", "fecha", "expediente", "es_perspectiva_genero"]
dataframe = None
total = 0
datos = respuesta.json()
if "items" in datos:
    dataframe = pd.json_normalize(datos["items"])
    total = datos["total"]
    dataframe = dataframe[columns]
    print(dataframe.head(), total)